Plik z modelami regresyjnymi. Modele testowałem na trzech rodzajach fingerprintów (MACCSFP, Klekota&Roth, haszowane) oraz na ich połączeniu. W zasadzie zawsze połączenie tych trzech zbiorów okazywało się najlepsze. Po wstępnej analize danych (z pliku data_analysis.ipynb) wydawało mi się, że dobrym pomysłem będzie usunięcie części najmniej znaczących zmiennych. Robiłem to poprzez wybieranie cech które pojawiają się w największej liczbie związków, oraz w drugim podejściu przy pomocy funkcji SelectKBest z sklearn. Okazało się jednak, że większa liczba zmiennych mimo wszystko poprawia wyniki. Modele korzystające tylko z małej części danych są jednak w stanie osiągać dość podobne wyniki, a przy tym czas wykonania jest zdecydowanie mniejszy. Pokazuje to także, że jesteśmy w stanie w ten sposób identyfikować jakąś cześć cech, która jest najbardziej odpowiedzialna za kardiotoksyczność związków. W notebooku przetestowałem sporo modeli płytkich oraz prostą sieć neuronową. Do oceny wyników używałem r2_score. Najlepszy wynik udało mi się uzyskać za pomocą SVR (~0.68 r^2), w przypadku sieci neuronowej było to ~0.62 r2. Dokładniejsze wyniki można prześledzić poniżej. Ze względu na to, że testowałem dośc dużo kombinacji modeli/danych/parametrów cały notebook liczy się dość długo. W razie własnych testów polecam wywoływać funkcje z ograniczoną liczbą zmiennych, wtedy powinno działać szybciej. Powinno dać się to zrobić przez usunięcie częsci parametrów w miejscu gdzie wywołuje główne funkcje.

In [1]:
import numpy as np

from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, LogisticRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score

from warnings import filterwarnings

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import History
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers import BatchNormalization

import matplotlib.pyplot as plt

import utils

In [2]:
seed = 1

In [3]:
def RidgeRegr(X, y, cv):
    params = {'alpha':np.hstack([np.linspace(1e-9, 2, 10), np.linspace(10, 1e9, 3), np.linspace(0.1, 2, 3), np.array([1.0])]),
              'max_iter':np.array([50000])}

    grid = GridSearchCV(Ridge(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [4]:
def LassoRegr(X, y, cv):
    params = {'alpha':np.hstack([np.linspace(1e-9, 2, 1), np.linspace(10, 1e9, 1), np.linspace(0.1, 2, 1), np.array([1.0])]),
              'max_iter':np.array([10000])}

    grid = GridSearchCV(Lasso(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [5]:
def ElasticNetRegr(X, y, cv):
    params = {'alpha':np.hstack([np.linspace(1e-9, 2, 3), np.linspace(10, 1e9, 3), np.linspace(0.1, 2, 1), np.array([1.0])]),
              'max_iter':np.array([10000])}
    
    grid = GridSearchCV(ElasticNet(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [6]:
def BayesianRidgeRegr(X, y, cv):
    params = {'alpha_1':np.hstack([np.linspace(1e-9, 2, 5), np.linspace(10, 1e9, 3), np.linspace(0.1, 2, 3), np.array([1e-6])]),
              'alpha_2':np.hstack([np.linspace(1e-9, 2, 5), np.linspace(10, 1e9, 3), np.linspace(0.1, 2, 3), np.array([1e-6])])}
    
    grid = GridSearchCV(BayesianRidge(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [7]:
def SGDRegressorRegr(X, y, cv):
    params = {'alpha':np.hstack([np.linspace(1e-9, 2, 10), np.linspace(10, 1e9, 3), np.linspace(0.1, 2, 3), np.array([0.0001])])}

    grid = GridSearchCV(SGDRegressor(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [8]:
def SVMRegr(X, y, cv):
    params = {'C':np.hstack([np.linspace(1e-9, 2, 1), np.linspace(10, 1e9, 1), np.linspace(0.1, 2, 1), np.array([1.0])])}
    grid = GridSearchCV(SVR(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [9]:
def DecisionTreeRegr(X, y, cv):
    params = {'ccp_alpha':np.hstack(np.array([0]))}
    grid = GridSearchCV(DecisionTreeRegressor(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [10]:
def KNeighborsRegr(X, y, cv):
    params = {'algorithm':np.array(['auto'])}
    grid = GridSearchCV(KNeighborsRegressor(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [11]:
def perform_regression(X, y, svm_only=False):
    
    model_functions = [
    ("Ridge", RidgeRegr),
    ("Lasso", LassoRegr),
    ("Elastic Net", ElasticNetRegr),
    ("Bayesian", BayesianRidgeRegr),
    ("SGD", SGDRegressorRegr),
    ("SVM", SVMRegr),
    ("Decision Tree", DecisionTreeRegr),
    ("K Neighbors", KNeighborsRegr),
    ]
    
    if svm_only:
        model_functions = [ ("SVM", SVMRegr) ]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

    kfold = KFold(n_splits=5, random_state=seed, shuffle=True)
    
    np.seterr(divide='ignore', invalid='ignore')
    filterwarnings('ignore')

    print(f"{'Regression model:'.ljust(19)}   R^2:")

    best_score = 0
    best_result = ("None", 0, {})

    for (name, function) in model_functions:
        model = function(X_train, y_train, kfold)
        prediction = model.best_estimator_.predict(X_test)
        score = r2_score(y_test, prediction)
        if (score > best_score):
            best_score = score
            best_result = (name, score, model.best_params_)

        print(f"{name.ljust(19)}   {str(round(score, 6)).ljust(9)}   {model.best_params_}")

    print(f"\nBest\n{best_result[0].ljust(19)}   {str(round(best_result[1], 6)).ljust(9)}   {best_result[2]}\n")
    

In [ ]:
for perc in [0.9, 0.8, 0.5, 0.3, 0.1]:
    df_hashed = utils.get_hashed_fingerprints(min_perc_used=perc)
    df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=perc)
    df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=perc)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=perc)
    dfs = [
        (df_hashed, "Hashed Extended Fingerprints"), 
        (df_maccsfp, "MACCSFP Fingerprints"), 
        (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing fetures that are present in at least {perc*100}% of substances.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        print(title, '\n')
        perform_regression(X, y)
        print()
    print('\n\n')

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 51)
Shape after removing outliers: (10396, 51)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 15)
Shape after removing outliers: (10396, 15)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 9)
Shape after removing outliers: (10396, 9)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 15)
Shape after removing outliers: (1039

DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 59)
Shape after removing outliers: (10396, 59)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 36)
Shape after removing outliers: (10396, 36)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 59)
Shape after removing outliers: (10396, 59)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 36)
Shape after removing outliers: (10396, 36)

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.cs

DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 161)
Shape after removing outliers: (10396, 161)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 118)
Shape after removing outliers: (10396, 118)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 161)
Shape after removing outliers: (10396, 161)

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 1007)
Shape after removing outliers: (10396, 1007)


Using fetures that are present in at l

Tutaj musiałem przerwać obliczenia ze względu na czas, wywołałem ponownie dla pozostałych parametrów, teraz już tylko dla połączonych fingerprintów, na których dostawałem najlepsze wyniki

In [ ]:
for perc in [0.1, 0.01, 0]:
#     df_hashed = utils.get_hashed_fingerprints(min_perc_used=perc)
#     df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=perc)
#     df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=perc)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=perc)
    dfs = [
#         (df_hashed, "Hashed Extended Fingerprints"), 
#         (df_maccsfp, "MACCSFP Fingerprints"), 
#         (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing fetures that are present in at least {perc*100}% of substances.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        print(title, '\n')
        perform_regression(X, y)
        print()
    print('\n\n')

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 118)
Shape after removing outliers: (10396, 118)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 161)
Shape after removing outliers: (10396, 161)

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 1007)
Shape after removing outliers: (10396, 1007)


Using fetures that are present in at least 10.0% of substances.

Mixed Fingerprints 

Regression model:     R^2:
Ridge                 0.472615    {'alpha': 10.0, 'max_iter': 50000}
Lasso                 0.458676    {'alpha

In [16]:
for perc in [0.01, 0]:
#     df_hashed = utils.get_hashed_fingerprints(min_perc_used=perc)
#     df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=perc)
#     df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=perc)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=perc)
    dfs = [
#         (df_hashed, "Hashed Extended Fingerprints"), 
#         (df_maccsfp, "MACCSFP Fingerprints"), 
#         (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing fetures that are present in at least {perc*100}% of substances.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        print(title, '\n')
        perform_regression(X, y, svm_only=True)
        print()
    print('\n\n')

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 138)
Shape after removing outliers: (10396, 138)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 619)
Shape after removing outliers: (10396, 619)

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 1008)
Shape after removing outliers: (10396, 1008)


Using fetures that are present in at least 1.0% of substances.

Mixed Fingerprints 

Regression model:     R^2:
SVM                   0.684308    {'C': 10.0}

Best
SVM                   0.684308    {'C': 10.0}





Preparin

Tutaj testuje metode SelectKBest do wyboru zmiennych. Wyniki są tutaj nieco lepsze od wcześniejszej metody.

In [ ]:
# perc values adjusted to compare with previous method
for perc in [0.0119, 0.02, 0.059, 0.133, 0.21, 0.29]:
    df_hashed = utils.get_hashed_fingerprints(min_perc_used=0)
    df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=0)
    df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=0)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=0)
    dfs = [
        (df_hashed, "Hashed Extended Fingerprints"), 
        (df_maccsfp, "MACCSFP Fingerprints"), 
        (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing custom feture selection with {perc*100}% of all features.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        select = int(X.shape[1] * perc)
        np.seterr(divide='ignore', invalid='ignore')
        X = SelectKBest(f_regression, k=select).fit_transform(X, y)
        print(title, '\n')
        perform_regression(X, y)
        print()
    print('\n\n')

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing outliers: (10396, 1025)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing

DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing outliers: (10396, 1025)


Using custom feture selection with 5.8999999999999995% of all features.

Hashed Extended Fingerprints 

Regression model:     R^2:
Ridge                 0.190791    {'alpha': 10.0, 'max_iter': 50000}
Lasso                 0.190229    {'alpha': 1e-09, 'max_iter': 10000}
Elastic Net           0.190229    {'alpha': 1e-09, 'max_iter': 10000}
Bayesian              0.192376    {'alpha_1': 1e-09, 'alpha_2': 10.0}
SGD                   0.169354    {'alpha': 0.0001}
SVM                   0.563535    {'C': 10.0}
Decision Tree         0.21992     {'ccp_alpha': 0}
K Neighbors           0.510373    {'algorithm': 'auto'}

Best
SVM                   0.563535    {'C': 10.0}


MACCSFP F

Ridge                 0.140177    {'alpha': 10.0, 'max_iter': 50000}
Lasso                 0.140371    {'alpha': 1e-09, 'max_iter': 10000}
Elastic Net           0.140371    {'alpha': 1e-09, 'max_iter': 10000}
Bayesian              0.140028    {'alpha_1': 10.0, 'alpha_2': 1e-09}
SGD                   0.131914    {'alpha': 0.0001}
SVM                   0.346262    {'C': 10.0}
Decision Tree         0.252092    {'ccp_alpha': 0}
K Neighbors           0.336015    {'algorithm': 'auto'}

Best
SVM                   0.346262    {'C': 10.0}


Klekota&Roth Fingerprints 

Regression model:     R^2:
Ridge                 0.490341    {'alpha': 2.0, 'max_iter': 50000}
Lasso                 0.474765    {'alpha': 1e-09, 'max_iter': 10000}
Elastic Net           0.47476     {'alpha': 1e-09, 'max_iter': 10000}
Bayesian              0.482737    {'alpha_1': 10.0, 'alpha_2': 1e-09}
SGD                   0.423243    {'alpha': 1e-09}
SVM                   0.670656    {'C': 10.0}
Decision Tree         0.388235  

Na koniec przetestowałem jeszcze prostą sieć neuronową

In [13]:
def perform_NN_regression(X, y):
    history = History()
    model = Sequential()

    size = X.shape[1]
    model.add(Dense(size//2+1, activation="relu",input_shape=(size,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(size//4+1, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(size//8+1, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(size//16+1, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(size//32+1, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.summary()


    early_stopping = EarlyStopping(patience=30, monitor="val_loss")
    model.compile(loss='mean_absolute_error', optimizer="Adam")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

    model.fit(X_train, y_train, validation_data= (X_test, y_test), batch_size=256, epochs=200, validation_split=0.2, callbacks=[early_stopping, history], verbose=0)
    prediction = model.predict(X_test)
    score = r2_score(y_test, prediction)
    print(f'\nFinal R^2 score:   {score}')
    print()
    

In [14]:
# perc values adjusted to compare with previous method
for perc in [0.0119, 0.02, 0.059, 0.133, 0.21, 0.29]:
    df_hashed = utils.get_hashed_fingerprints(min_perc_used=0)
    df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=0)
    df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=0)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=0)
    dfs = [
        (df_hashed, "Hashed Extended Fingerprints"), 
        (df_maccsfp, "MACCSFP Fingerprints"), 
        (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing custom feture selection with {perc*100}% of all features.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        select = int(X.shape[1] * perc)
        np.seterr(divide='ignore', invalid='ignore')
        X = SelectKBest(f_regression, k=select).fit_transform(X, y)
        print(title, '\n')
        perform_NN_regression(X, y)
        print()
    print('\n\n')

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing outliers: (10396, 1025)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing


Final R^2 score:   0.030716218988498745


Mixed Fingerprints 

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_96 (Dense)            (None, 36)                2592      
                                                                 
 batch_normalization_80 (Bat  (None, 36)               144       
 chNormalization)                                                
                                                                 
 dropout_80 (Dropout)        (None, 36)                0         
                                                                 
 dense_97 (Dense)            (None, 18)                666       
                                                                 
 batch_normalization_81 (Bat  (None, 18)               72        
 chNormalization)                                                
                                                       


Final R^2 score:   -0.026936440494289116


Klekota&Roth Fingerprints 

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_114 (Dense)           (None, 49)                4802      
                                                                 
 batch_normalization_95 (Bat  (None, 49)               196       
 chNormalization)                                                
                                                                 
 dropout_95 (Dropout)        (None, 49)                0         
                                                                 
 dense_115 (Dense)           (None, 25)                1250      
                                                                 
 batch_normalization_96 (Bat  (None, 25)               100       
 chNormalization)                                                
                                               


Final R^2 score:   0.010383078029607362


MACCSFP Fingerprints 

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_132 (Dense)           (None, 5)                 50        
                                                                 
 batch_normalization_110 (Ba  (None, 5)                20        
 tchNormalization)                                               
                                                                 
 dropout_110 (Dropout)       (None, 5)                 0         
                                                                 
 dense_133 (Dense)           (None, 3)                 18        
                                                                 
 batch_normalization_111 (Ba  (None, 3)                12        
 tchNormalization)                                               
                                                     


Final R^2 score:   0.5301164405291321





Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing outliers: (10396, 1025)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after


Final R^2 score:   0.6072325046799534


Mixed Fingerprints 

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_168 (Dense)           (None, 403)               324415    
                                                                 
 batch_normalization_140 (Ba  (None, 403)              1612      
 tchNormalization)                                               
                                                                 
 dropout_140 (Dropout)       (None, 403)               0         
                                                                 
 dense_169 (Dense)           (None, 202)               81608     
                                                                 
 batch_normalization_141 (Ba  (None, 202)              808       
 tchNormalization)                                               
                                                         


Final R^2 score:   -0.013972179573121135


Klekota&Roth Fingerprints 

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_186 (Dense)           (None, 511)               521731    
                                                                 
 batch_normalization_155 (Ba  (None, 511)              2044      
 tchNormalization)                                               
                                                                 
 dropout_155 (Dropout)       (None, 511)               0         
                                                                 
 dense_187 (Dense)           (None, 256)               131072    
                                                                 
 batch_normalization_156 (Ba  (None, 256)              1024      
 tchNormalization)                                               
                                               


Final R^2 score:   0.4850757616188788


MACCSFP Fingerprints 

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_204 (Dense)           (None, 25)                1225      
                                                                 
 batch_normalization_170 (Ba  (None, 25)               100       
 tchNormalization)                                               
                                                                 
 dropout_170 (Dropout)       (None, 25)                0         
                                                                 
 dense_205 (Dense)           (None, 13)                338       
                                                                 
 batch_normalization_171 (Ba  (None, 13)               52        
 tchNormalization)                                               
                                                       


Final R^2 score:   0.43774142885860845







In [15]:
for perc in reversed([0, 0.01, 0.1, 0.3, 0.5, 0.8, 0.9]):
    df_hashed = utils.get_hashed_fingerprints(min_perc_used=perc)
    df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=perc)
    df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=perc)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=perc)
    dfs = [
        (df_hashed, "Hashed Extended Fingerprints"), 
        (df_maccsfp, "MACCSFP Fingerprints"), 
        (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing fetures that are present in at least {perc*100}% of substances.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        print(title, '\n')
        perform_NN_regression(X, y)
        print()
    print('\n\n')

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 51)
Shape after removing outliers: (10396, 51)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 15)
Shape after removing outliers: (10396, 15)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 9)
Shape after removing outliers: (10396, 9)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 15)
Shape after removing outliers: (1039


Final R^2 score:   -10.789249898487117


Mixed Fingerprints 

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 36)                2628      
                                                                 
 batch_normalization_15 (Bat  (None, 36)               144       
 chNormalization)                                                
                                                                 
 dropout_15 (Dropout)        (None, 36)                0         
                                                                 
 dense_19 (Dense)            (None, 18)                666       
                                                                 
 batch_normalization_16 (Bat  (None, 18)               72        
 chNormalization)                                                
                                                         


Final R^2 score:   -10.789249898487117


Klekota&Roth Fingerprints 

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 7)                 105       
                                                                 
 batch_normalization_30 (Bat  (None, 7)                28        
 chNormalization)                                                
                                                                 
 dropout_30 (Dropout)        (None, 7)                 0         
                                                                 
 dense_37 (Dense)            (None, 3)                 24        
                                                                 
 batch_normalization_31 (Bat  (None, 3)                12        
 chNormalization)                                                
                                                  


Final R^2 score:   0.32007536149258176


MACCSFP Fingerprints 

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 29)                1711      
                                                                 
 batch_normalization_45 (Bat  (None, 29)               116       
 chNormalization)                                                
                                                                 
 dropout_45 (Dropout)        (None, 29)                0         
                                                                 
 dense_55 (Dense)            (None, 14)                420       
                                                                 
 batch_normalization_46 (Bat  (None, 14)               56        
 chNormalization)                                                
                                                       


Final R^2 score:   0.3973103893275761





Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 655)
Shape after removing outliers: (10396, 655)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 80)
Shape after removing outliers: (10396, 80)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 70)
Shape after removing outliers: (10396, 70)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: 


Final R^2 score:   0.055933103508097215


Mixed Fingerprints 

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 401)               322003    
                                                                 
 batch_normalization_75 (Bat  (None, 401)              1604      
 chNormalization)                                                
                                                                 
 dropout_75 (Dropout)        (None, 401)               0         
                                                                 
 dense_91 (Dense)            (None, 200)               80400     
                                                                 
 batch_normalization_76 (Bat  (None, 200)              800       
 chNormalization)                                                
                                                       


Final R^2 score:   0.15548729740158285


Klekota&Roth Fingerprints 

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_108 (Dense)           (None, 80)                12880     
                                                                 
 batch_normalization_90 (Bat  (None, 80)               320       
 chNormalization)                                                
                                                                 
 dropout_90 (Dropout)        (None, 80)                0         
                                                                 
 dense_109 (Dense)           (None, 40)                3240      
                                                                 
 batch_normalization_91 (Bat  (None, 40)               160       
 chNormalization)                                                
                                                 


Final R^2 score:   0.5655981119947822


MACCSFP Fingerprints 

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_126 (Dense)           (None, 68)                9384      
                                                                 
 batch_normalization_105 (Ba  (None, 68)               272       
 tchNormalization)                                               
                                                                 
 dropout_105 (Dropout)       (None, 68)                0         
                                                                 
 dense_127 (Dense)           (None, 34)                2346      
                                                                 
 batch_normalization_106 (Ba  (None, 34)               136       
 tchNormalization)                                               
                                                       


Final R^2 score:   0.6191569557104494





Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing outliers: (10396, 1025)

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing outliers: (10396, 4861)

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing outliers: (10396, 167)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after


Final R^2 score:   0.5382226773563157


Mixed Fingerprints 

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_162 (Dense)           (None, 3025)              18304275  
                                                                 
 batch_normalization_135 (Ba  (None, 3025)             12100     
 tchNormalization)                                               
                                                                 
 dropout_135 (Dropout)       (None, 3025)              0         
                                                                 
 dense_163 (Dense)           (None, 1512)              4575312   
                                                                 
 batch_normalization_136 (Ba  (None, 1512)             6048      
 tchNormalization)                                               
                                                         

Testowanie SVR (najlepszego modelu) z większą liczbą hiperparametrów.

In [14]:
def SVMRegr(X, y, cv):
    params = {'C':np.hstack([np.linspace(1e-9, 1e-3, 2), np.linspace(1e3, 1e9, 2), np.linspace(2, 10, 2), np.array([1.0])]),
              'epsilon':np.hstack([np.linspace(1e-9, 1e-3, 2), np.linspace(1e3, 1e9, 2), np.linspace(1, 10, 2), np.array([0.1])])}
    grid = GridSearchCV(SVR(), params, cv=cv, return_train_score=True)
    grid.fit(X, y)
    return grid

In [15]:
for perc in [0.01]:
#     df_hashed = utils.get_hashed_fingerprints(min_perc_used=perc)
#     df_maccsfp = utils.get_MACCSFP_fingerprints(min_perc_used=perc)
#     df_klekota = utils.get_KlekotaRoth_fingerprints(min_perc_used=perc)
    df_mixed = utils.get_mixed_fingerprints(min_perc_used=perc)
    dfs = [
#         (df_hashed, "Hashed Extended Fingerprints"), 
#         (df_maccsfp, "MACCSFP Fingerprints"), 
#         (df_klekota, "Klekota&Roth Fingerprints"), 
        (df_mixed, "Mixed Fingerprints")
    ]
    
    print(f'\nUsing fetures that are present in at least {perc*100}% of substances.\n')
    for df, title in dfs:
        X = df.drop('IC50', axis=1)
        y = df['IC50']
        print(title, '\n')
        perform_regression(X, y, svm_only=True)
        print()
    print('\n\n')

Preparing files for mixed fingerprints.

Preparing (ready_sets/cardiotoxicity_hERG_MACCSFP.csv) file.
DataFrame base shape: (11504, 167)
Shape after removing wrong values: (10635, 167)
Shape after removing least used features: (10635, 138)
Shape after removing outliers: (10396, 138)

Preparing (ready_sets/cardiotoxicity_hERG_KlekFP.csv) file.
DataFrame base shape: (11504, 4861)
Shape after removing wrong values: (10635, 4861)
Shape after removing least used features: (10635, 619)
Shape after removing outliers: (10396, 619)

Preparing (ready_sets/cardiotoxicity_hERG_ExtFP.csv) file.
DataFrame base shape: (11504, 1025)
Shape after removing wrong values: (10635, 1025)
Shape after removing least used features: (10635, 1008)
Shape after removing outliers: (10396, 1008)


Using fetures that are present in at least 1.0% of substances.

Mixed Fingerprints 

Regression model:     R^2:
SVM                   0.684308    {'C': 10.0, 'epsilon': 0.1}

Best
SVM                   0.684308    {'C': 10.